In [2]:
import json
import os
import re
import string
import sys
from functools import reduce
from pathlib import Path

import matplotlib.pyplot as plt
import metapub
import nltk
import numpy as np
import pandas as pd
import plotly.express as px
import pubmed_parser as pp
import requests
import seaborn as sns
import spacy
from Bio import Entrez
from docling.document_converter import DocumentConverter
from dotenv import load_dotenv
from metapub import PubMedFetcher
from metapub.findit import FindIt
from tqdm import tqdm
from unidecode import unidecode
from wordcloud import WordCloud

load_dotenv()
_ = os.getenv("NCBI_API_KEY")
Entrez.email = os.getenv("email")

In [5]:
! black .

Skipping .ipynb files as Jupyter dependencies are not installed.
You can fix this by running ``pip install "black[jupyter]"``
No Python files are present to be formatted. Nothing to do 😴


### New Query

In [4]:
notebook_path = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_path, '..')))

from extractors import pmids_extractor, pubmed_extractor, findIt_extractor

In [7]:
keyword = '''
(CFTR[Title/Abstract] OR "CFTR gene"[Title/Abstract]) AND ("Cystic fibrosis"[Title/Abstract]) AND (mutant*[Title/Abstract] OR variant*[Title/Abstract] OR
mutat*[Title/Abstract]) AND (("free energy"[Title/Abstract] OR "Gibbs free energy"[Title/Abstract] OR ΔG[Title/Abstract] OR "delta G"[Title/Abstract] OR 
"kcal/mol"[Title/Abstract] OR "kj/mol"[Title/Abstract] OR "melting temperature"[Title/Abstract] OR Tm[Title/Abstract] OR thermostab*[Title/Abstract] OR 
stabil*[Title/Abstract] OR destabili*[Title/Abstract])) AND ("protein stability"[Title/Abstract] OR "thermodynamics"[Title/Abstract]) NOT (review[Publication Type] OR 
"molecular dynamics"[Title/Abstract] OR predict*[Title/Abstract])'''

pmid_list = pmids_extractor(keyword)

Extraindo do PubMed: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 905/905 [10:35<00:00,  1.42it/s]


In [13]:
print(len(pmid_list))
pmid_list = set(pmid_list)
print(pmid_list)
print(len(pmid_list))

11
{'32934006', '28655774', '28087700', '29903914', '36834620', '19502384', '28510592', '35327663', '30668920', '8830258', '25799099'}
11


In [19]:
pmid_list = list(pmid_list)

In [21]:
def pubmed_extractor(pmids_list, output_path="data/pubmed_output.jsonl", key = _):
    fetch = PubMedFetcher()
    not_found_pmids = []

    with open(output_path, "a", encoding="utf-8") as f_out:
        for pmid in tqdm(pmids_list, desc="Extraindo do PubMed"):
            try:
                article = fetch.article_by_pmid(pmid)
                xml = article.xml
                parsed = pp.parse_medline_xml(
                    xml,
                    year_info_only=False,
                    nlm_category=False,
                    author_list=False,
                    reference_list=False,
                )
                for entry in parsed:
                    entry["url"] = findIt_extractor(pmid, _)
                    f_out.write(json.dumps(entry) + "\n")
            except Exception as e:
                tqdm.write(f"PMID {pmid}: {e}")
                not_found_pmids.append(pmid)
                continue

    # Salva os PMIDs com erro
    if not_found_pmids:
        with open("data/pmids_nao_encontrados.json", "w", encoding="utf-8") as f:
            json.dump(not_found_pmids, f, indent=2)

    print(f"Extração finalizada. Total com erro: {len(not_found_pmids)}")

In [22]:
pubmed_extractor(pmid_list, output_path="../data/cystic_fibrosis_data.jsonl", key = _)

Extraindo do PubMed: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 11/11 [00:18<00:00,  1.68s/it]

Extração finalizada. Total com erro: 0


In [24]:
df = pd.read_json('../data/cystic_fibrosis/cystic_fibrosis_data.jsonl', lines=True)
#df.to_excel('../data/cystic_fibrosis/cystic_fibrosis_data.xlsx')
df = df.drop_duplicates('pmid')
df.shape

(11, 23)

In [26]:
df

,title,issue,pages,abstract,journal,authors,pubdate,pmid,mesh_terms,publication_types,...,references,delete,affiliations,pmc,other_id,medline_ta,nlm_unique_id,issn_linking,country,url
0,Analysis of the localization of STE6/CFTR chim...,19(5),1007-17,The use of yeast as a model system to study ma...,Molecular microbiology,Paddon|C|C|;Loayza|D|D|;Vangelista|L|L|;Solari...,1996-03,8830258,D018528:ATP-Binding Cassette Transporters; D00...,D016428:Journal Article; D013487:Research Supp...,...,,False,"Cell Biology Unit, Glaxo Wellcome Medicines Re...",,,Mol Microbiol,8712028,0950-382X,England,Dont Found
1,Functional stability of rescued delta F508 cys...,42(3),363-72,The most common mutation in the cystic fibrosi...,American journal of respiratory cell and molec...,Jurkuvenaite|Asta|A|;Chen|Lan|L|;Bartoszewski|...,2010-03,19502384,D002460:Cell Line; D016764:Cell Polarity; D019...,D016428:Journal Article; D052061:Research Supp...,...,9922375;18389279;1699669;9792704;7523390;75538...,False,"Department of Cell Biology, University of Alab...",,,Am J Respir Cell Mol Biol,8917225,1044-1549,United States,http://europepmc.org/backend/ptpmcrender.fcgi?...
3,Design and characterization of a membrane prot...,10(3),e0120253,Accurate measurement of membrane protein stabi...,PloS one,Nadeau|Vincent G|VG|;Gao|Anqi|A|;Deber|Charles...,2015,25799099,D000595:Amino Acid Sequence; D019005:Cystic Fi...,D016428:Journal Article; D013485:Research Supp...,...,8566543;18573083;23180855;23745835;12324439;20...,False,"Division of Molecular Structure & Function, Re...",,,PLoS One,101285081,1932-6203,United States,http://europepmc.org/backend/ptpmcrender.fcgi?...
5,Two Small Molecules Restore Stability to a Sub...,292(9),3706-3719,Cystic fibrosis (CF) is caused by mutations th...,The Journal of biological chemistry,Meng|Xin|X|;Wang|Yiting|Y|;Wang|Xiaomeng|X|;Wr...,2017-03-03,28087700,D000627:Aminophenols; D000631:Aminopyridines; ...,D016428:Journal Article; D013485:Research Supp...,...,27189798;25607428;2475911;16554808;22675668;23...,False,"From the Faculty of Biology, Medicine and Heal...",,,J Biol Chem,2985121R,0021-9258,United States,http://europepmc.org/backend/ptpmcrender.fcgi?...
7,Alteration of protein function by a silent pol...,15(5),e2000779,Synonymous single nucleotide polymorphisms (sS...,PLoS biology,Kirchner|Sebastian|S|;Cai|Zhiwei|Z|;Rauscher|R...,2017-05,28510592,D019005:Cystic Fibrosis Transmembrane Conducta...,D016428:Journal Article; D013485:Research Supp...,...,22955616;18516229;16418745;23417065;23417067;2...,False,"Biochemistry, Institute of Biochemistry and Bi...",,,PLoS Biol,101183755,1544-9173,United States,http://europepmc.org/backend/ptpmcrender.fcgi?...
8,Stabilization of a nucleotide-binding domain o...,292(34),14147-14164,Characterization of the second nucleotide-bind...,The Journal of biological chemistry,Vernon|Robert M|RM|;Chong|P Andrew|PA|;Lin|Hon...,2017-08-25,28655774,D000251:Adenosine Triphosphatases; D000255:Ade...,D016428:Journal Article; D013485:Research Supp...,...,2475911;1973824;10792060;1712984;9305845;17161...,False,"From the Program in Molecular Medicine, Hospit...",,,J Biol Chem,2985121R,0021-9258,United States,http://europepmc.org/backend/ptpmcrender.fcgi?...
10,Ligand binding to a remote site thermodynamica...,293(46),17685-17704,Many disease-causing mutations impair protein ...,The Journal of biological chemistry,Wang|Chi|C|;Aleksandrov|Andrei A|AA|;Yang|Zhen...,2018-11-16,29903914,D000255:Adenosine Triphosphate; D001665:Bindin...,D016428:Journal Article; D052061:Research Supp...,...,2475911;22675668;1695717;15729345;17036051;117...,False,From the Departments of Biological Sciences.;D...,,,J Biol Chem,2985121R,0021-9258,United States,http://europepmc.org/backend/ptpmcrender.fcgi?...
12,A defective flexible loop contributes to the p...,33(4),5126-5142,People with the genetic disease cystic fibrosi...,FASEB journal : official publication of the Fe...,Chen|Xinying|X|;Zhu|Siyu|S|;Zhenin|Michael|M|;...,2019-04,30668

In [31]:
path_pdf = "../data/cystic_fibrosis/pdfs"
for i, (url, pmid) in tqdm(enumerate(zip(df['url'], df['pmid'])), total=len(df), desc="Downloading PDFs"):
    if url != "Dont Found":
        # Sanitize o DOI para ser um nome de arquivo válido
        #pmid_filename = doi.replace("/", "_").replace(":", "_")
        nome_arquivo = os.path.join(path_pdf, f"{pmid}.pdf")

        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(nome_arquivo, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)

In [8]:
def extract_entities(model, document_text, document_name):
    nlp = model.load()

    # Gerenciar o EntityRuler
    if "entity_ruler" in nlp.pipe_names:
        ruler = nlp.get_pipe("entity_ruler")
        ruler.clear()
    else:
        ruler = nlp.add_pipe("entity_ruler", before="ner")

    # Labels personalizadas
    custom_labels = {"MUTATION", "TEMPERATURE", "PH", "METODO"}
    
    # Padrões corrigidos
    patterns = [
        # MUTATION
        {"label": "MUTATION", "pattern": [{"TEXT": {"REGEX": "^[A-Z][0-9]+[A-Z]$"}}]},
        {"label": "MUTATION", "pattern": [{"LOWER": "mutation"}, {"TEXT": {"REGEX": "^[A-Z][0-9]+[A-Z]$"}}]},
        
        # PH
        {"label": "PH", "pattern": [{"LOWER": "ph"}, {"TEXT": {"REGEX": r"^[+-]?\d+(\.\d+)?$"}}]},
        
        # TEMPERATURE
        {"label": "TEMPERATURE", "pattern": [
            {"TEXT": {"REGEX": r"^\d+(\.\d+)?$"}},
            {"TEXT": {"REGEX": r"^°$"}},
            {"TEXT": {"REGEX": r"^[CFK]$"}}
        ]},
        {"label": "TEMPERATURE", "pattern": [
            {"TEXT": {"REGEX": r"^\d+(\.\d+)?°[CFK]$"}}
        ]},
    
        # METODO
         {"label": "METODO", "pattern": [
        {"LOWER": "differential"},
        {"LOWER": "scanning"},
        {"LOWER": "calorimetry"},
        {"TEXT": "(", "OP": "?"},
        {"LOWER": "dsc", "OP": "?"},
        {"TEXT": ")", "OP": "?"}
    ]},
    # Apenas DSC (com ou sem parênteses)
    {"label": "METODO", "pattern": [
        {"TEXT": "(", "OP": "?"},
        {"LOWER": "dsc"},
        {"TEXT": ")", "OP": "?"}
    ]},

    # CD completo com ou sem parênteses
    {"label": "METODO", "pattern": [
        {"LOWER": "circular"},
        {"LOWER": "dichroism"},
        {"TEXT": "(", "OP": "?"},
        {"LOWER": "cd", "OP": "?"},
        {"TEXT": ")", "OP": "?"},
        {"LOWER": "spectroscopy"}
    ]},
    # CD spectroscopy (abreviado)
    {"label": "METODO", "pattern": [
        {"TEXT": "(", "OP": "?"},
        {"LOWER": "cd"},
        {"TEXT": ")", "OP": "?"},
        {"LOWER": "spectroscopy"}
    ]},
    # Apenas CD (com ou sem parênteses)
    {"label": "METODO", "pattern": [
        {"TEXT": "(", "OP": "?"},
        {"LOWER": "cd"},
        {"TEXT": ")", "OP": "?"}
    ]}
        ]

    ruler.add_patterns(patterns)

    # Processamento do texto
    doc = nlp(document_text)

    # Extração sem duplicatas
    seen = set()
    entity = []
    label = []

    for ent in doc.ents:
        if ent.label_ in custom_labels and ent.text not in seen:
            entity.append(ent.text)
            label.append(ent.label_)
            seen.add(ent.text)

    # Regex para termos termodinâmicos
    delta_pattern = re.compile(r'''
        (?:Δ|delta|Delta)\s*[GT]|𝚫\s*[GT]?                       
        (?:\s*[:=~]\s*([-\d.]+)\s*(kcal/mol|kJ/mol|°C|K)?)?      
        (?:.*?(Fig(?:ure)?\s*[\dS]+)?)?     
        ''', flags=re.IGNORECASE | re.VERBOSE)

    delta_matches = []
    for match in delta_pattern.finditer(document_text):
        termo = match.group("symbol").upper()
        valor = match.group("value")
        unidade = match.group("unit") or ""
        delta_matches.append({
            "termo": f"Δ{termo}",
            "valor": valor,
            "unidade": unidade
        })

    return {
        "pmid": document_name,
        "entity": entity,
        "label": label,
        "termos_termodinamicos": delta_matches,
        "tem_termo_termodinamico": "Sim" if delta_matches else "Não"
    }

class SpacyModel:
    def load(self):
        return spacy.load("en_core_web_sm")

In [9]:
source_pdfs = Path("../data/cystic_fibrosis/pdfs")
#pasta_output = Path("../data/cystic_fibrosis/md")

pdfs = sorted(source_pdfs.glob("*.pdf"))

md_list = []

for pdf in pdfs:  
    input_dir = str(pdf)
    converter = DocumentConverter()
    result = converter.convert(input_dir)
    verify = extract_entities(SpacyModel(),result.document.export_to_markdown(), pdf.name)
    md_list.append(verify)

/home/grati/miniconda3/envs/drugdiscovery/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/grati/miniconda3/envs/drugdiscovery/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/grati/miniconda3/envs/drugdiscovery/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/grati/miniconda3/envs/drugdiscovery/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.war

In [30]:
rows = []

# Vamos construir uma tabela linha por linha
for item in md_list:
    pmid = item['pmid']
    entities = item['entity']
    labels = item['label']
    termos = item.get('termos_termodinamicos', [])
    tem_termo = item.get('tem_termo_termodinamico', "Não")
    
    # Associa cada entidade à sua label correta
    for ent, lab in zip(entities, labels):
        rows.append({
            'pmid': pmid,
            'label': lab,
            'entity': ent,
            'termos_termodinamicos': termos,
            'tem_termo_termodinamico': tem_termo
        })

# Criar DataFrame estruturado corretamente
df = pd.DataFrame(rows)

# Agora pivotar de forma limpa: cada label vira uma coluna
df_grouped = df.groupby(['pmid', 'label'])['entity'].apply(list).unstack(fill_value=[]).reset_index()

# Adiciona os campos extras (termodinâmica)
df_termos = df[['pmid', 'tem_termo_termodinamico']].drop_duplicates()

# Junta tudo
df_final = pd.merge(df_grouped, df_termos, on='pmid', how='left')

df_final

,pmid,METODO,MUTATION,PH,TEMPERATURE,tem_termo_termodinamico
0,19502384.pdf,[],[P2Y],[pH 7.4],[],Não
1,25799099.pdf,"[Circular dichroism spectroscopy, CD spectrosc...","[M5S, S45A, E44Q]","[pH 8, pH 7.5, pH 2, pH 9]","[37°C, 4°C]",Não
2,28087700.pdf,[differential scanning calorimetry],"[mutation G551D, G551D, G1349D, A561E]","[pH 7.3, pH 7.2, pH 8.0]","[37 °C, 10 °C, 23 °C, 27 °C, 80 °C, 5 °C, 60 °...",Não
3,28510592.pdf,[],"[T2562G, S1A, G1584A, G2280A, S1B, S1C, S1E, S...","[pH 7.3, pH 7.5, pH 8.5, pH 9.0]",[],Não
4,28655774.pdf,"[DSC, differential scanning calorimetry, diffe...","[M5S, G1349D, G551D, G1244E, S1251N, S1255P, m...","[pH 7.5, pH 7.6, pH 7.2]","[40 °C, 4 °C, 49 °C, 10 °C, 3 °C, 30 °C, 7 °C,...",Não
5,29903914.pdf,"[DSC, differential scanning calorimetry, diffe...","[G646C, C592L, X4C, V510D, P2Y]","[pH 7.5, pH 6]","[37 °C, 3 °C, 3.5 °C, 0.3 °C, 2.5 °C, 3.3 °C, ...",Sim
6,32934006.pdf,[],"[G551D, mutation R1070W, I152L, R1070W, D579G,...",[pH 7.4],"[37 °C, 28 °C, 37° C]",Não
7,35327663.pdf,[],"[R1070W, mutation E1371Q, S1A, S1C, S1E, E1371...",[],[],Não
8,36834620.pdf,[],"[H139R, G194R, H199Y, W361R, L558S, A559E, L57...",[pH 7],[],Não


In [62]:
# Primeiro, limpamos os DOIs
for item in md_list[1:-1]:
    item['pmid'] = item['doi'].replace('.pdf', '')

# Criamos um DataFrame a partir da lista
df_termos = pd.DataFrame(md_list)
df_termos = df_termos[1:]
df_termos['pmid'] = df_termos['pmid'].astype('int64')

In [63]:
df_termos

,doi,tem_termo_termodinamico,pmid
1,19502384.pdf,Sim,19502384
2,25799099.pdf,Não,25799099
3,28087700.pdf,Sim,28087700
4,28510592.pdf,Não,28510592
5,28655774.pdf,Sim,28655774
6,29903914.pdf,Sim,29903914
7,32934006.pdf,Não,32934006
8,35327663.pdf,Sim,35327663
9,36834620.pdf,Sim,36834620


In [72]:
data = pd.read_csv('../data/cystic_fibrosis/articles.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   key        608 non-null    object 
 1   title      608 non-null    object 
 2   year       587 non-null    float64
 3   month      29 non-null     float64
 4   day        12 non-null     float64
 5   journal    510 non-null    object 
 6   issn       230 non-null    object 
 7   volume     297 non-null    float64
 8   issue      100 non-null    float64
 9   pages      267 non-null    object 
 10  authors    608 non-null    object 
 11  url        607 non-null    object 
 12  language   510 non-null    object 
 13  publisher  0 non-null      float64
 14  location   176 non-null    object 
 15  abstract   592 non-null    object 
 16  notes      608 non-null    object 
 17  doi        537 non-null    object 
 18  keywords   573 non-null    object 
 19  pubmed_id  0 non-null      float64
 20  pmc_id    

In [75]:
data_pubmed = data_pubmed[['doi', 'pmid', 'affiliations', 'country']]
data_pubmed

data = pd.merge(data,data_pubmed,how='left',on=['doi'])

In [76]:
data['location'] = data['location'].fillna(data['affiliations'])
data['pmid'] = data['pmid'].fillna(data['pmc_id'])

In [77]:
pd.set_option('display.max_colwidth', None)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   key           608 non-null    object 
 1   title         608 non-null    object 
 2   year          587 non-null    float64
 3   month         29 non-null     float64
 4   day           12 non-null     float64
 5   journal       510 non-null    object 
 6   issn          230 non-null    object 
 7   volume        297 non-null    float64
 8   issue         100 non-null    float64
 9   pages         267 non-null    object 
 10  authors       608 non-null    object 
 11  url           607 non-null    object 
 12  language      510 non-null    object 
 13  publisher     0 non-null      float64
 14  location      321 non-null    object 
 15  abstract      592 non-null    object 
 16  notes         608 non-null    object 
 17  doi           537 non-null    object 
 18  keywords      573 non-null    

In [78]:
data = data[['title','doi' ,'year', 'month', 'journal', 'url', 'location', 'pmid', 'country']]

In [79]:
data.shape

(608, 9)

In [80]:
data_null = data[data.doi.notnull() & data.location.isnull()]
data_null.info()

<class 'pandas.core.frame.DataFrame'>
Index: 243 entries, 0 to 583
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     243 non-null    object 
 1   doi       243 non-null    object 
 2   year      243 non-null    float64
 3   month     1 non-null      float64
 4   journal   203 non-null    object 
 5   url       243 non-null    object 
 6   location  0 non-null      object 
 7   pmid      0 non-null      object 
 8   country   0 non-null      object 
dtypes: float64(2), object(7)
memory usage: 19.0+ KB


In [81]:
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc, FullDoc
from elsapy.elssearch import ElsSearch

load_dotenv()
apikey = os.getenv("X_ELS_APIKey")
insttoken = os.getenv("X_ELS_Insttoken")

client = ElsClient(apikey)
client.inst_token = insttoken

2025-05-23 18:22:35 grati elsapy.elsclient[2537748] INFO Module loaded.
2025-05-23 18:22:35 grati elsapy.elsentity[2537748] INFO Module loaded.
2025-05-23 18:22:35 grati elsapy.elsdoc[2537748] INFO Module loaded.
2025-05-23 18:22:35 grati elsapy.utils[2537748] INFO Module loaded.
2025-05-23 18:22:35 grati elsapy.elssearch[2537748] INFO Module loaded.


In [82]:
# Lista para armazenar os resultados
rows = []

# Itera sobre os DOIs onde location está ausente
for doi in data_null['doi']:
    try:
        doc_srch = ElsSearch(doi, "scopus")
        doc_srch.execute(client)

        if doc_srch.results:
            item = doc_srch.results[0]  # Apenas o primeiro resultado
            affs = item.get("affiliation", [])
            country = affs[0].get("affiliation-country") if affs else None
            rows.append({"doi": doi, "country": country})
        else:
            rows.append({"doi": doi, "country": None})
    except Exception as e:
        print(f"Erro ao buscar DOI {doi}: {e}")
        rows.append({"doi": doi, "country": None})

# Cria o DataFrame com os resultados
data_scopus = pd.DataFrame(rows)

2025-05-23 18:22:44 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.ejphar.2024.176771
2025-05-23 18:22:46 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:22:47 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.pan.2024.07.015
2025-05-23 18:22:48 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:22:49 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.urology.2023.02.017
2025-05-23 18:22:50 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:22:51 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.pan.2024.07.009
2025-05-23 18:22:52 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:22:53 grati els

Erro ao buscar DOI 10.1016/S2213-2600(20)30510-5: HTTP 400 Error from https://api.elsevier.com/content/search/scopus?query=10.1016%2FS2213-2600%2820%2930510-5
and using headers {'X-ELS-APIKey': '18cea8004848f339d14dd509a16e753e', 'User-Agent': 'elsapy-v0.5.1', 'Accept': 'application/json', 'X-ELS-Insttoken': 'aa8087d5506a067fa3e1c2761628f450'}:
{"service-error":{"status":{"statusCode":"INVALID_INPUT","statusText":"Error translating query"}}}


2025-05-23 18:23:50 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.biocel.2021.105976
2025-05-23 18:23:51 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:23:52 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1152%2FAJPLUNG.00137.2020
2025-05-23 18:23:52 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:23:53 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.jcf.2020.07.015
2025-05-23 18:23:54 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:23:55 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.tiv.2021.105253
2025-05-23 18:23:56 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:23:57 grati elsapy

Erro ao buscar DOI 10.1016/S2213-2600(18)30460-0: HTTP 400 Error from https://api.elsevier.com/content/search/scopus?query=10.1016%2FS2213-2600%2818%2930460-0
and using headers {'X-ELS-APIKey': '18cea8004848f339d14dd509a16e753e', 'User-Agent': 'elsapy-v0.5.1', 'Accept': 'application/json', 'X-ELS-Insttoken': 'aa8087d5506a067fa3e1c2761628f450'}:
{"service-error":{"status":{"statusCode":"INVALID_INPUT","statusText":"Error translating query"}}}


2025-05-23 18:24:15 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.rmed.2020.106027
2025-05-23 18:24:15 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:24:16 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1080%2F17476348.2020.1682998
2025-05-23 18:24:17 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:24:18 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2Fj.jcf.2019.09.018
2025-05-23 18:24:19 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:24:20 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1002%2Fhumu.23741
2025-05-23 18:24:20 grati elsapy.utils[2537748] INFO Converting prism:coverDate
2025-05-23 18:24:21 grati elsapy.elscl

...

2025-05-23 18:29:27 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2FS1569-1993%2820%2930346-5


Erro ao buscar DOI 10.1016/S1569-1993(20)30346-5: HTTP 400 Error from https://api.elsevier.com/content/search/scopus?query=10.1016%2FS1569-1993%2820%2930346-5
and using headers {'X-ELS-APIKey': '18cea8004848f339d14dd509a16e753e', 'User-Agent': 'elsapy-v0.5.1', 'Accept': 'application/json', 'X-ELS-Insttoken': 'aa8087d5506a067fa3e1c2761628f450'}:
{"service-error":{"status":{"statusCode":"INVALID_INPUT","statusText":"Error translating query"}}}


2025-05-23 18:29:29 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1016%2FS1569-1993%2820%2930369-6


Erro ao buscar DOI 10.1016/S1569-1993(20)30369-6: HTTP 400 Error from https://api.elsevier.com/content/search/scopus?query=10.1016%2FS1569-1993%2820%2930369-6
and using headers {'X-ELS-APIKey': '18cea8004848f339d14dd509a16e753e', 'User-Agent': 'elsapy-v0.5.1', 'Accept': 'application/json', 'X-ELS-Insttoken': 'aa8087d5506a067fa3e1c2761628f450'}:
{"service-error":{"status":{"statusCode":"INVALID_INPUT","statusText":"Error translating query"}}}


2025-05-23 18:29:30 grati elsapy.elsclient[2537748] INFO Sending GET request to https://api.elsevier.com/content/search/scopus?query=10.1021%2Fbi702209s
2025-05-23 18:29:31 grati elsapy.utils[2537748] INFO Converting prism:coverDate


In [83]:
data_scopus = data_scopus.rename(columns={'country': 'affiliation_country'})

In [84]:
data_scopus.shape

(243, 2)

In [85]:
data_affiliation_scopus = pd.merge(data, data_scopus, how='left',on=['doi'])
data_affiliation_scopus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                679 non-null    object 
 1   doi                  608 non-null    object 
 2   year                 658 non-null    float64
 3   month                29 non-null     float64
 4   journal              549 non-null    object 
 5   url                  678 non-null    object 
 6   location             360 non-null    object 
 7   pmid                 196 non-null    object 
 8   country              196 non-null    object 
 9   affiliation_country  229 non-null    object 
dtypes: float64(2), object(8)
memory usage: 53.2+ KB


In [86]:
data_affiliation_scopus = data_affiliation_scopus.drop_duplicates('title')
data_affiliation_scopus = data_affiliation_scopus[['title', 'doi', 'year', 'month','journal', 'url', 'location', 'pmid', 'country', 'affiliation_country']]

data_affiliation_scopus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 607 entries, 0 to 678
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                607 non-null    object 
 1   doi                  536 non-null    object 
 2   year                 586 non-null    float64
 3   month                29 non-null     float64
 4   journal              509 non-null    object 
 5   url                  606 non-null    object 
 6   location             320 non-null    object 
 7   pmid                 196 non-null    object 
 8   country              196 non-null    object 
 9   affiliation_country  178 non-null    object 
dtypes: float64(2), object(8)
memory usage: 52.2+ KB


In [87]:
data_affiliation_scopus['location'] = data_affiliation_scopus['location'].fillna(data_affiliation_scopus['affiliation_country'])
data_affiliation_scopus.info()

<class 'pandas.core.frame.DataFrame'>
Index: 607 entries, 0 to 678
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                607 non-null    object 
 1   doi                  536 non-null    object 
 2   year                 586 non-null    float64
 3   month                29 non-null     float64
 4   journal              509 non-null    object 
 5   url                  606 non-null    object 
 6   location             483 non-null    object 
 7   pmid                 196 non-null    object 
 8   country              196 non-null    object 
 9   affiliation_country  178 non-null    object 
dtypes: float64(2), object(8)
memory usage: 52.2+ KB


In [88]:
data_affiliation_scopus.head()

,title,doi,year,month,journal,url,location,pmid,country,affiliation_country
0,Localization and function of humanized F508del-CFTR in mouse intestine following activation of serum glucocorticoid kinase 1 and Trikafta,10.1016/j.ejphar.2024.176771,2024.0,NaN,European Journal of Pharmacology,https://www.scopus.com/inward/record.uri?eid=2-s2.0-85197319328&doi=10.1016%2fj.ejphar.2024.176771&partnerID=40&md5=1c76818fdd0b3a61215c9dfed67ebadf,United States,NaN,NaN,United States
1,L1077P CFTR pathogenic variant function rescue by Elexacaftor–Tezacaftor–Ivacaftor in cystic fibrosis patient-derived air–liquid interface (ALI) cultures and organoids: in vitro guided personalized therapy of non-F508del patients,10.1186/s12931-023-02516-0,2023.0,NaN,Respiratory Research,https://www.scopus.com/inward/record.uri?eid=2-s2.0-85172256705&doi=10.1186%2fs12931-023-02516-0&partnerID=40&md5=ac29de89df9cf5cd0f2ec1cb77dc2a91,"Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Department of Experimental Medicine, Sapienza University of Rome, Rome, Italy.;Department of Experimental Medicine, Sapienza University of Rome, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Department of Molecular Medicine and Medical Biotechnologies, University of Naples Federico II, Naples, Italy.;Department of Molecular Medicine and Medical Biotechnologies, University of Naples Federico II, Naples, Italy.;Department of Molecular Medicine and Medical Biotechnologies, University of Naples Federico II, Naples, Italy.;Confocal Microscopy Unit, Core Facilities, Istituto Superiore di Sanità, Rome, Italy.;Dipartimento di Medicina e Chirurgia Traslazionale, Università Cattolica del Sacro Cuore, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Cystic Fibrosis Reference Center of Lazio Region, AOU Policlinico Umberto I, Rome, Italy.;Department of Molecular Medicine and Medical Biotechnologies, University of Naples Federico II, Naples, Italy.;Department of Experimental Medicine, Sapienza University of Rome, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy. adriana.eramo@iss.it.",37674160,England,NaN
2,"Long-Term Safety and Efficacy of Elexacaftor/Tezacaftor/Ivacaftor in Children Aged >6 Years with Cystic Fibrosis and at Least One F508del Allele A Phase 3, Open-Label Clinical Trial",10.1164/rccm.202301-0021OC,2023.0,NaN,American Journal of Respiratory and Critical Care Medicine,https://www.scopus.com/inward/record.uri?eid=2-s2.0-85164233720&doi=10.1164%2frccm.202301-0021OC&partnerID=40&md5=fb1946b9e74c8d01c4fd7c205be7ed58,"Queensland Children's Hospital, South Brisbane, Queensland, Australia.;Ann & Robert H. Lurie Children's Hospital of Chicago, Chicago, Illinois.;Children's Health Ireland, Dublin, Ireland.;Oregon Health & Sciences University, Portland, Oregon.;University of Toronto, Toronto, Ontario, Canada.;University of British Columbia, Vancouver, British Columbia, Canada.;University of North Carolina at Chapel Hill, Chapel Hill, North Carolina.;University Hospitals Cleveland Medical Center, Cleveland, Ohio.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;National Heart and Lung Institute, Imperial College London, London, United Kingdom; and.",37154609,United States,NaN
3,Reply to: Cyst

In [89]:
from geotext import GeoText

def extract_first_country_from_list_or_str(entry):
    if pd.isnull(entry):
        return None
    if isinstance(entry, str):
        entry = [entry]
    for item in entry:
        places = GeoText(item)
        if places.countries:
            return places.countries[0]
    return None

In [90]:
data_affiliation_scopus['country_from_location'] = data_affiliation_scopus['location'].apply(extract_first_country_from_list_or_str)
data_affiliation_scopus.head()

,title,doi,year,month,journal,url,location,pmid,country,affiliation_country,country_from_location
0,Localization and function of humanized F508del-CFTR in mouse intestine following activation of serum glucocorticoid kinase 1 and Trikafta,10.1016/j.ejphar.2024.176771,2024.0,NaN,European Journal of Pharmacology,https://www.scopus.com/inward/record.uri?eid=2-s2.0-85197319328&doi=10.1016%2fj.ejphar.2024.176771&partnerID=40&md5=1c76818fdd0b3a61215c9dfed67ebadf,United States,NaN,NaN,United States,United States
1,L1077P CFTR pathogenic variant function rescue by Elexacaftor–Tezacaftor–Ivacaftor in cystic fibrosis patient-derived air–liquid interface (ALI) cultures and organoids: in vitro guided personalized therapy of non-F508del patients,10.1186/s12931-023-02516-0,2023.0,NaN,Respiratory Research,https://www.scopus.com/inward/record.uri?eid=2-s2.0-85172256705&doi=10.1186%2fs12931-023-02516-0&partnerID=40&md5=ac29de89df9cf5cd0f2ec1cb77dc2a91,"Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Department of Experimental Medicine, Sapienza University of Rome, Rome, Italy.;Department of Experimental Medicine, Sapienza University of Rome, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Department of Molecular Medicine and Medical Biotechnologies, University of Naples Federico II, Naples, Italy.;Department of Molecular Medicine and Medical Biotechnologies, University of Naples Federico II, Naples, Italy.;Department of Molecular Medicine and Medical Biotechnologies, University of Naples Federico II, Naples, Italy.;Confocal Microscopy Unit, Core Facilities, Istituto Superiore di Sanità, Rome, Italy.;Dipartimento di Medicina e Chirurgia Traslazionale, Università Cattolica del Sacro Cuore, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy.;Cystic Fibrosis Reference Center of Lazio Region, AOU Policlinico Umberto I, Rome, Italy.;Department of Molecular Medicine and Medical Biotechnologies, University of Naples Federico II, Naples, Italy.;Department of Experimental Medicine, Sapienza University of Rome, Rome, Italy.;Department of Oncology and Molecular Medicine, Istituto Superiore di Sanità, Rome, Italy. adriana.eramo@iss.it.",37674160,England,NaN,Italy
2,"Long-Term Safety and Efficacy of Elexacaftor/Tezacaftor/Ivacaftor in Children Aged >6 Years with Cystic Fibrosis and at Least One F508del Allele A Phase 3, Open-Label Clinical Trial",10.1164/rccm.202301-0021OC,2023.0,NaN,American Journal of Respiratory and Critical Care Medicine,https://www.scopus.com/inward/record.uri?eid=2-s2.0-85164233720&doi=10.1164%2frccm.202301-0021OC&partnerID=40&md5=fb1946b9e74c8d01c4fd7c205be7ed58,"Queensland Children's Hospital, South Brisbane, Queensland, Australia.;Ann & Robert H. Lurie Children's Hospital of Chicago, Chicago, Illinois.;Children's Health Ireland, Dublin, Ireland.;Oregon Health & Sciences University, Portland, Oregon.;University of Toronto, Toronto, Ontario, Canada.;University of British Columbia, Vancouver, British Columbia, Canada.;University of North Carolina at Chapel Hill, Chapel Hill, North Carolina.;University Hospitals Cleveland Medical Center, Cleveland, Ohio.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;Vertex Pharmaceuticals Incorporated, Boston, Massachusetts.;National Heart and Lung Institute, Imperial College London, London, United Kingdom; and.",3